In [6]:
%pip install seaborn
import seaborn as sns
import pandas as pd

# import training & testing datasets that we want to work with (provided by problem)
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# quick preview of some possible data types/values
train_df.info()
train_df.head()


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# determine if there's any dupes to deal with up front
print('train_df # dup. records:', train_df.duplicated().sum())
print('test_df # dup. records:', test_df.duplicated().sum())


train_df # dup. records: 0
test_df # dup. records: 0


In [8]:
# split dataset into logical chunks so we can show off Feast features from multiple sources
survivors_df = train_df[train_df.columns[:2]]
demographics_df =train_df[train_df.columns[2:6]]
genealogy_df = train_df[train_df.columns[6:8]]
trip_df = train_df[train_df.columns[8:]]
target_df = pd.DataFrame(train_df["Survived"], columns=["Survived"])

print("survivors subset now consists of {0}".format(survivors_df.columns.values))
print("demographics subset now consists of {0}".format(demographics_df.columns.values))
print("genealogy subset now consists of {0}".format(genealogy_df.columns.values))
print("trip subset now consists of {0}".format(trip_df.columns.values))


survivors subset now consists of ['PassengerId' 'Survived']
demographics subset now consists of ['Pclass' 'Name' 'Sex' 'Age']
genealogy subset now consists of ['SibSp' 'Parch']
trip subset now consists of ['Ticket' 'Fare' 'Cabin' 'Embarked']


In [9]:
# append a timestamp as we aren't really tracking any specific times in our data, but Feast wants one
# Creating timestamps for the data
timestamps = pd.date_range(
    end=pd.Timestamp.now(),
    periods=len(survivors_df),
    freq='D').to_frame(name="event_timestamp", index=False)

# add to each source
survivors_df = pd.concat(objs=[survivors_df, timestamps], axis=1)
demographics_df = pd.concat(objs=[demographics_df, timestamps], axis=1)
genealogy_df = pd.concat(objs=[genealogy_df, timestamps], axis=1)
trip_df = pd.concat(objs=[trip_df, timestamps], axis=1)


In [10]:
# append the PassengerId across the list (we could generate arbitrary values, but we'll use what's provided)
passenger_ids = survivors_df['PassengerId']

demographics_df = pd.concat(objs=[demographics_df, passenger_ids], axis=1)
genealogy_df = pd.concat(objs=[genealogy_df, passenger_ids], axis=1)
trip_df = pd.concat(objs=[trip_df, passenger_ids], axis=1)

survivors_df.to_parquet(path='data/survivors_source.parquet')
demographics_df.to_parquet(path='data/demographics_source.parquet')
genealogy_df.to_parquet(path='data/genealogy_source.parquet')
trip_df.to_parquet(path='data/trip_source.parquet')

# peek at one just to verify what went in
trip_df.info()
trip_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Ticket           891 non-null    object        
 1   Fare             891 non-null    float64       
 2   Cabin            204 non-null    object        
 3   Embarked         889 non-null    object        
 4   event_timestamp  891 non-null    datetime64[ns]
 5   PassengerId      891 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 41.9+ KB


,Ticket,Fare,Cabin,Embarked,event_timestamp,PassengerId
0,A/5 21171,7.2500,NaN,S,2021-07-07 17:11:52.360260,1
1,PC 17599,71.2833,C85,C,2021-07-08 17:11:52.360260,2
2,STON/O2. 3101282,7.9250,NaN,S,2021-07-09 17:11:52.360260,3
3,113803,53.1000,C123,S,2021-07-10 17:11:52.360260,4
4,373450,8.0500,NaN,S,2021-07-11 17:11:52.360260,5
